In [2]:
import ee
import geemap
from utils import *

initialize()
config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder

def fc_to_image(fc, property, scale = 1000):
    # Convert the feature collection to an image
    return fc.reduceToImage(
            properties = [property],
            reducer = ee.Reducer.first()
        ).reproject(
            crs = 'EPSG:4326',  # or match your source CRS
            scale = scale
        ).rename(property)


def export_image(image, description):
    task = ee.batch.Export.image.toDrive(
        image = image,
        description = description,
        fileNamePrefix = description,
        region = roi,
        scale = image.projection().nominalScale(),
        maxPixels = 1e13,
        crs = 'EPSG:4326',
        fileFormat = 'GeoTIFF'
    )
    task.start()

In [8]:
field = ee.FeatureCollection("projects/amazon-forest-regrowth/assets/field")


biome = ee.Image("projects/amazon-forest-regrowth/assets/categorical").select("biome").eq(1)


sampled_points = biome.selfMask().sampleRegions(
    collection=field,
    scale=30,  # or use the appropriate resolution of your biome image
    geometries=True  # Include geometries for each pixel
)

# task = ee.batch.Export.table.toDrive(
#     collection=sampled_points,
#     description='field_points_amazon',
#     fileNamePrefix='field_points_amazon',
#     fileFormat='SHP'
# )
# task.start()

map = geemap.Map()
map.addLayer(sampled_points, {}, "Sampled Points")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [2]:
quarters = ee.Image("projects/amazon-forest-regrowth/assets/quarters_ecoreg_biomass").select("quarter_biomass")

task = ee.batch.Export.image.toDrive(
    image=quarters,
    description='quarters',
    fileNamePrefix='quarters',
    region=roi,
    scale=1000)

task.start()

In [3]:
lulc = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
            .select([f"classification_{year}" for year in config.range_1985_2020])
            .byte()
            .rename([str(year) for year in config.range_1985_2020]))

pastureland = lulc.select("2020").eq(15).unmask(0).rename("pastureland")

map = geemap.Map()
map.addLayer(pastureland, {"palette": ["green", "yellow"]}, "Pastureland 2020")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Export asymptotes for Figure 2

In [ ]:
nearest_mature = ee.Image(f"{data_folder}/nearest_mature").selfMask()

quarters_ecoreg_biomass = ee.Image(f"{data_folder}/quarters_ecoreg_biomass").select("quarter_biomass").selfMask()

biomes = ee.Image(f"{data_folder}/categorical").select("biome")
biomes_mask = biomes.eq(1).rename("biome_mask").selfMask()

# nearest_mature = ee.Image(f"{data_folder}/nearest_mature").selfMask()


# export_image(nearest_mature, "nearest_mature")
# export_image(quarters_ecoreg_biomass, "quarters_ecoreg_biomass")
# export_image(biomes_mask, "biome_mask")

In [8]:
pixels_to_sample.projection()

## Export predictions for Figure 4

In [ ]:
# image = ee.Image(f"{data_folder}/mapbiomas_2020").select("age")

image = ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_secondary_vegetation_age_v1").select("secondary_vegetation_age_2020")


biomes = ee.Image(f"{config.data_folder}/categorical").select("biome")


biomes = biomes.eq(1).selfMask()

pixels_to_sample = biomes.reduceResolution(
    ee.Reducer.first(), maxPixels=65536
    ).reproject(
    crs = image.projection().getInfo()['crs'],
    crsTransform = image.projection().getInfo()['transform']
    ).updateMask(image)

image_scale = round(pixels_to_sample.projection().nominalScale().getInfo())
closest_multiple = round(1000 / image_scale) * image_scale

# First, sample locations based only on the age band
grid = geemap.create_grid(pixels_to_sample.geometry(), closest_multiple, pixels_to_sample.projection())


# Predictions for 2050
pred_lag_2050_fc = ee.FeatureCollection(f"{data_folder}/results/pred_2050_secondary")

# pred_lag_2050 = fc_to_image(pred_lag_2050_fc, "pred").reproject(crs = pixels_to_sample.projection().getInfo()['crs'] , crsTransform = pixels_to_sample.projection().atScale(990).getInfo()['transform'])

pred_lag_2050 = pred_lag_2050_fc.reduceToImage(properties=['pred'], reducer=ee.Reducer.first()).reproject(crs = pixels_to_sample.projection().getInfo()['crs'] , crsTransform = pixels_to_sample.projection().atScale(990).getInfo()['transform'])

secondary_area = ee.Image("projects/amazon-forest-regrowth/assets/secondary_area_1km")

# # Aggregate the high-resolution pixels into the 10 km grid
# pred_lag_2050_10k = pred_lag_2050.reduceResolution(
#     reducer = ee.Reducer.median(),
#     maxPixels = 65535,
#     bestEffort = True
# ).reproject(
#     crs = 'EPSG:4326',
#     crsTransform=age.projection().atScale(990).getInfo()['transform'],
# ).rename("pred_lag_2050_10k")

vis_params = {'min': 0, 'max': 75, 'palette': ["#003f5c", "#2f4b7c", "#665191", "#a05195",
    "#d45087", "#f95d6a", "#ff7c43", "#ffa600", "#ffc300", "#ffda6a"]}

# map = geemap.Map()
# # map.addLayer(pred_lag_2050_fc, {}, "pred_lag_2050_fc")
# # map.addLayer(grid, {}, "grid")
# # map.addLayer(pred_lag_2050, vis_params, "biomass_2050")
# map.addLayer(pred_lag_2050, vis_params, "pred_lag_2050")
# # map.addLayer(secondary_area, {}, "secondary_area")
# map.addLayer(pred_lag_2050_interpolated, vis_params, "pred_lag_2050_interpolated")
# # map.add_colorbar(vis_params, label = "biomass (Mg/ha)", layer_name = "biomass")
# map

# export_image(pred_lag_2050_10k, "pred_2050_secondary")

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [3]:
image = ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_secondary_vegetation_age_v1").select("secondary_vegetation_age_2020")

age = ee.Image(f"{data_folder}/mapbiomas_2020")


vis_params = {'min': 0, 'max': 35, 'palette': ["#003f5c", "#2f4b7c", "#665191", "#a05195",
    "#d45087", "#f95d6a", "#ff7c43", "#ffa600", "#ffc300", "#ffda6a"]}


map = geemap.Map()
map.addLayer(image, vis_params, "age_original")
map.addLayer(age, vis_params, "age_new")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [4]:
lulc = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
            .select([f"classification_{year}" for year in config.range_1985_2020])
            .byte()
            .rename([str(year) for year in config.range_1985_2020]))

map.addLayer(lulc, {}, "LULC")

In [22]:

area_manually_selected = map.draw_last_feature

tiles = area_manually_selected.geometry().coveringGrid(image.projection(), 10000)  # 1000km tiles
map.addLayer(tiles, {}, "tiles")

In [23]:
# Create a grid of tiles

def sum_tile(feature):
    tile_sum = pred_age.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=feature.geometry(),
        scale=300,
        maxPixels=1e9
    )
    return feature.set(tile_sum)

tile_sums = tiles.map(sum_tile)
total = tile_sums.aggregate_sum('pred')
total



# total = image.reduceRegion(
#     reducer=ee.Reducer.sum(),
#     geometry=image.geometry(),
#     scale=300,  # Adjust based on your needs
#     maxPixels=1e13,
#     bestEffort=True,
#     tileScale=4
# ).get('pred').getInfo()

KeyboardInterrupt: 

In [ ]:
pred_lag_2050_fc = ee.FeatureCollection("projects/amazon-forest-regrowth/assets/results/pred_2050_pastureland_all")
pred_lag_2050 = fc_to_image(pred_lag_2050_fc, "pred", scale=100)

# Aggregate the high-resolution pixels into the 10 km grid
pred_lag_2050 = pred_lag_2050.reduceResolution(
    reducer = ee.Reducer.median(),
    maxPixels = 65535,
    bestEffort = True
).reproject(
    crs = 'EPSG:4326',
    scale = 10000
).rename("pred_lag_2050_10k")

export_image(pred_lag_2050, "pred_lag_2050_pastureland_all_10k_raw")

In [ ]:
mature_biomass = ee.Image(f"{data_folder}/mature_biomass")

# Aggregate the high-resolution pixels into the 10 km grid
mature_biomass = mature_biomass.reduceResolution(
    reducer = ee.Reducer.median(),
    maxPixels = 65535,
    bestEffort = True
).reproject(
    crs = 'EPSG:4326',
    scale = 10000
).rename("mature_biomass")

export_image(mature_biomass, "mature_biomass")

In [7]:

# Predictions for 2050
pred_lag_2050_fc = ee.FeatureCollection(f"{data_folder}/total_pred_lag_2050_secondary")
# multiply the value of pred by 1000
pred_lag_2050 = fc_to_image(pred_lag_2050_fc, "pred", scale=100)

# multiply value by 1000
# pred_lag_2050 = pred_lag_2050.multiply(1000)

# Aggregate the high-resolution pixels into the 10 km grid
pred_lag_2050_10k = pred_lag_2050.reduceResolution(
    reducer = ee.Reducer.median(),
    maxPixels = 65535,
    bestEffort = True
).reproject(
    crs = 'EPSG:4326',
    scale = 10000
).rename("pred_lag_2050_10k")


export_image(pred_lag_2050_10k, "total_pred_lag_2050_secondary")

In [ ]:
grid = ee.FeatureCollection("projects/amazon-forest-regrowth/assets/grid_1k_amazon_secondary")

map = geemap.Map()
map.addLayer(grid, {}, "grid")
map

## Export Extended Data Maps

In [ ]:
# Absolute error
relative_growth = ee.FeatureCollection(f"{data_folder}/results/relative_growth")
relative_growth = fc_to_image(relative_growth, "percentage")

vis_params = {'min': 40, 'max': 60, 'palette': ['yellow', 'blue']}

map = geemap.Map(basemap = 'CartoDB.Positron')
map.addLayer(relative_growth, vis_params, "relative_growth")
map.add_colorbar(vis_params, label = "Percentage of asymptote", layer_name = "relative_growth")
map



Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# Absolute error
absolute_error = ee.FeatureCollection(f"{data_folder}/results/absolute_error")
absolute_error = fc_to_image(absolute_error, "error")

# Percent error
percent_error = ee.FeatureCollection(f"{data_folder}/results/percent_error")
percent_error = fc_to_image(percent_error, "error")

vis_params = {'min': -100, 'max': 100, 'palette': ['red', 'blue']}

map = geemap.Map(basemap = 'CartoDB.Positron')
map.addLayer(absolute_error, vis_params, "absolute_error")
map.addLayer(percent_error, vis_params, "percent_error")
map.add_colorbar(vis_params, label = "Absolute error", layer_name = "absolute_error")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…